In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
import seaborn as sns

pd.set_option("display.max_columns", None)

In [2]:
year = 2017
election_round = "first_round"

In [3]:
# Read full data
if year == 2022:
    df = pd.read_excel(f"data/France/{year}_{election_round}.xlsx")
    df["polling_id"] = df["Libellé de la commune"].astype(str) + "-" + df["Code du b.vote"].astype(str)
    
elif year == 2017:
    df = pd.read_csv(f"data/France/{year}_{election_round}.txt", delimiter=";", encoding="latin-1")
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    
    df["polling_id"] = df["level_1"].astype(str) + "-" + df["level_3"].astype(str) + "-" +\
        df["level_5"].astype(str) + "-" + df["level_22"].astype(str)
    
    if election_round == "runoff":
        candidates = {
            "level_25": "Emmanuel MACRON",
            "level_32": "Marine LE PEN"
        }
        
    else:
        candidates = {
            "level_25": "Nicolas DUPONT-AIGNAN",
            "level_32": "Marine LE PEN",
            "level_39": "Emmanuel MACRON",
            "level_46": "Benoît HAMON",
            "level_53": "Nathalie ARTHAUD",
            "level_60": "Philippe POUTOU",
            "level_67": "Jacques CHEMINADE",
            "level_74": "Jean LASSALLE",
            "level_81": "Jean-Luc MÉLENCHON",
            "level_88": "François ASSELINEAU",
            "level_95": "François FILLON"
        }
        
    df = df.rename(columns=candidates)
    df_filtered = df[["polling_id"] + list(candidates.values())]

df.head()

C:\Users\cnavarreteliz\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_20,level_21,level_22,level_23,level_24,Nicolas DUPONT-AIGNAN,level_26,level_27,level_28,level_29,level_30,level_31,Marine LE PEN,level_33,level_34,level_35,level_36,level_37,level_38,Emmanuel MACRON,level_40,level_41,level_42,level_43,level_44,level_45,Benoît HAMON,level_47,level_48,level_49,level_50,level_51,level_52,Nathalie ARTHAUD,level_54,level_55,level_56,level_57,level_58,level_59,Philippe POUTOU,level_61,level_62,level_63,level_64,level_65,level_66,Jacques CHEMINADE,level_68,level_69,level_70,level_71,level_72,level_73,Jean LASSALLE,level_75,level_76,level_77,level_78,level_79,level_80,Jean-Luc MÉLENCHON,level_82,level_83,level_84,level_85,level_86,level_87,François ASSELINEAU,level_89,level_90,level_91,level_92,level_93,level_94,François FILLON,level_96,level_97,polling_id
0,1,Ain,4,4ème circonscription,1,L'Abergement-Clémenciat,1.0,598,92,"15,38",506,"84,62",2,"0,33","0,40",9,"1,51","1,78",495,"82,78","97,83",1,M,DUPONT-AIGNAN,Nicolas,34,"5,69","6,87",2,F,LE PEN,Marine,126,"21,07","25,45",3,M,MACRON,Emmanuel,119,"19,90","24,04",4,M,HAMON,Benoît,29,"4,85","5,86",5,F,ARTHAUD,Nathalie,4,"0,67","0,81",6,M,POUTOU,Philippe,4,"0,67","0,81",7,M,CHEMINADE,Jacques,2,"0,33","0,40",8,M,LASSALLE,Jean,2,"0,33","0,40",9,M,MÉLENCHON,Jean-Luc,59,"9,87","11,92",10,M,ASSELINEAU,François,6,"1,00","1,21",11,M,FILLON,François,110,"18,39","22,22",Ain-4ème circonscription-L'Abergement-Clémenci...
1,1,Ain,5,5ème circonscription,2,L'Abergement-de-Varey,1.0,209,25,"11,96",184,"88,04",6,"2,87","3,26",2,"0,96","1,09",176,"84,21","95,65",1,M,DUPONT-AIGNAN,Nicolas,6,"2,87","3,41",2,F,LE PEN,Marine,48,"22,97","27,27",3,M,MACRON,Emmanuel,37,"17,70","21,02",4,M,HAMON,Benoît,13,"6,22","7,39",5,F,ARTHAUD,Nathalie,2,"0,96","1,14",6,M,POUTOU,Philippe,2,"0,96","1,14",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,0,"0,00","0,00",9,M,MÉLENCHON,Jean-Luc,33,"15,79","18,75",10,M,ASSELINEAU,François,1,"0,48","0,57",11,M,FILLON,François,34,"16,27","19,32",Ain-5ème circonscription-L'Abergement-de-Varey-M
2,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,1.0,1116,233,"20,88",883,"79,12",17,"1,52","1,93",6,"0,54","0,68",860,"77,06","97,40",1,M,DUPONT-AIGNAN,Nicolas,58,"5,20","6,74",2,F,LE PEN,Marine,233,"20,88","27,09",3,M,MACRON,Emmanuel,156,"13,98","18,14",4,M,HAMON,Benoît,47,"4,21","5,47",5,F,ARTHAUD,Nathalie,6,"0,54","0,70",6,M,POUTOU,Philippe,15,"1,34","1,74",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,6,"0,54","0,70",9,M,MÉLENCHON,Jean-Luc,216,"19,35","25,12",10,M,ASSELINEAU,François,7,"0,63","0,81",11,M,FILLON,François,116,"10,39","13,49",Ain-5ème circonscription-Ambérieu-en-Bugey-M
3,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,2.0,1128,256,"22,70",872,"77,30",19,"1,68","2,18",3,"0,27","0,34",850,"75,35","97,48",1,M,DUPONT-AIGNAN,Nicolas,42,"3,72","4,94",2,F,LE PEN,Marine,234,"20,74","27,53",3,M,MACRON,Emmanuel,183,"16,22","21,53",4,M,HAMON,Benoît,51,"4,52","6,00",5,F,ARTHAUD,Nathalie,5,"0,44","0,59",6,M,POUTOU,Philippe,16,"1,42","1,88",7,M,CHEMINADE,Jacques,1,"0,09","0,12",8,M,LASSALLE,Jean,8,"0,71","0,94",9,M,MÉLENCHON,Jean-Luc,175,"15,51","20,59",10,M,ASSELINEAU,François,11,"0,98","1,29",11,M,FILLON,François,124,"10,99","14,59",Ain-5ème circonscription-Ambérieu-en-Bugey-M
4,1,Ain,5,5ème circonscription,4,Ambérieu-en-Bugey,3.0,1116,227,"20,34",889,"79,66",11,"0,99","1,24",4,"0,36","0,45",874,"78,32","98,31",1,M,DUPONT-AIGNAN,Nicolas,47,"4,21","5,38",2,F,LE PEN,Marine,218,"19,53","24,94",3,M,MACRON,Emmanuel,191,"17,11","21,85",4,M,HAMON,Benoît,53,"4,75","6,06",5,F,ARTHAUD,Nathalie,4,"0,36","0,46",6,M,POUTOU,Philippe,12,"1,08","1,37",7,M,CHEMINADE,Jacques,0,"0,00","0,00",8,M,LASSALLE,Jean,11,"0,99","1,26",9,M,MÉLENCHON,Jean-Luc,182,"16,31","20,82",10,M,ASSELINEAU,François,7,"0,63","0,80",11,M,FILLON,François,149,"13,35","17,05",Ain-5ème circonscription-Ambé

In [4]:
if year == 2022:

    if election_round == "runoff":
        df = df.rename(columns={
            "Voix": "Emmanuel MACRON",
            "Unnamed: 32": "Marine LE PEN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Emmanuel MACRON",
            "Marine LE PEN", 
        ]]
    else:
        df = df.rename(columns={
            "Voix": "Nathalie ARTHAUD",
            "Unnamed: 32": "Fabien ROUSSEL",
            "Unnamed: 39": "Emmanuel MACRON",
            "Unnamed: 46": "Jean LASSALLE",
            "Unnamed: 53": "Marine LE PEN",
            "Unnamed: 60": "Éric ZEMMOUR",
            "Unnamed: 67": "Jean-Luc MÉLENCHON",
            "Unnamed: 74": "Anne HIDALGO",
            "Unnamed: 81": "Yannick JADOT",
            "Unnamed: 88": "Valérie PÉCRESSE",
            "Unnamed: 95": "Philippe POUTOU",
            "Unnamed: 102": "Nicolas DUPONT-AIGNAN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Nathalie ARTHAUD", 
            "Fabien ROUSSEL", 
            "Emmanuel MACRON",
            "Jean LASSALLE", 
            "Marine LE PEN", 
            "Éric ZEMMOUR", 
            "Jean-Luc MÉLENCHON", 
            "Anne HIDALGO",
            "Yannick JADOT",
            "Valérie PÉCRESSE",
            "Philippe POUTOU",
            "Nicolas DUPONT-AIGNAN"
        ]]

In [5]:
df_filtered = df_filtered.melt(
    id_vars=["polling_id"], 
    var_name="Candidate", 
    value_name="Votes"
)

# df_filtered["polling_id"] = df_filtered["Libellé de la commune"].astype(str) + "-" + df_filtered["Code du b.vote"].astype(str)

df_filtered.head()

,polling_id,Candidate,Votes
0,Ain-4ème circonscription-L'Abergement-Clémenci...,Nicolas DUPONT-AIGNAN,34
1,Ain-5ème circonscription-L'Abergement-de-Varey-M,Nicolas DUPONT-AIGNAN,6
2,Ain-5ème circonscription-Ambérieu-en-Bugey-M,Nicolas DUPONT-AIGNAN,58
3,Ain-5ème circonscription-Ambérieu-en-Bugey-M,Nicolas DUPONT-AIGNAN,42
4,Ain-5ème circonscription-Ambérieu-en-Bugey-M,Nicolas DUPONT-AIGNAN,47


In [6]:
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["Votes"].rank("min", ascending=False).astype(int)

In [7]:
tt = df_filtered.groupby(["polling_id", "Candidate"]).agg({"Votes": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "Candidate", "rate"]]

In [8]:
df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "Candidate"])
df_filtered = df_filtered.rename(columns={"Votes": "value", "Candidate": "candidate"})

In [9]:
df_filtered.to_csv(f"data_output/France/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [10]:
if year == 2017:

    df_location = df[["polling_id", "level_0", "level_1", "level_2", "level_3", "level_4", "level_5", "level_22", "level_7", "level_8"]]

    df_location = df_location.rename(columns={
        "level_0": "department_id",
        "level_1": "department",
        "level_2": "circunscription_id",
        "level_3": "circunscription",
        "level_4": "commune_id",
        "level_5": "commune",
        "level_22": "polling_place",
        "level_7": "electors",
        "level_8": "abstentions"
    })
    
elif year == 2022:
    df_location = df[["Code du département", "Libellé du département", "Code de la circonscription", "Libellé de la circonscription",
       "Code de la commune", "Libellé de la commune", "Code du b.vote", "Inscrits", "Abstentions", "polling_id"]]

    df_location = df_location.rename(columns={
        "Code du département": "department_id",
        "Libellé du département": "department",
        "Code de la circonscription": "circunscription_id",
        "Libellé de la circonscription": "circunscription",
        "Code de la commune": "commune_id",
        "Libellé de la commune": "commune",
        "Code du b.vote": "polling_place",
        "Inscrits": "electors",
        "Abstentions": "abstentions"
    })

df_location.to_csv(f"data_output/France/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)

In [11]:


# df_location.to_csv(f"data_output/France/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)

In [12]:
# df_gini = pd.read_excel("https://www.insee.fr/fr/statistiques/fichier/5371235/RPM2021-F18.xlsx", sheet_name="figure 2", header=2)
# df_gini = df_gini.head(101)
# df_gini.head()

In [13]:
# measure = "Weight"

# data1 = df_1st_round.groupby(["Code du département", "Candidate"]).agg({measure: "mean"}).reset_index()

# vmax = data1[measure].max()
# vmin = data1[measure].min()

# tmp = pd.merge(gdf, data1, left_on="code", right_on="Code du département")
# matplotlib.rcParams["figure.dpi"] = 200

# tmp = tmp[tmp["Candidate"] == "Marine LE PEN"].copy()

# fig, ax = plt.subplots()

# tmp.plot(ax=ax, edgecolor="#222222", cmap="plasma", linewidth=0.3, column=measure, legend=True)
# # color=aa["color"], 

# ax.axis("off")
# fig.tight_layout()

In [14]:
# data2 = df_1st_round.groupby(["Code du département", "Candidate"]).agg({"Weight": "std"}).reset_index()
# data2 = data2.groupby(["Code du département"]).agg({"Weight": "mean"}).reset_index()

# dd = pd.merge(data2, df_gini, left_on="Code du département", right_on="Code")
# dd[["Weight", "Taux de pauvreté (en\xa0%)"]].corr()

In [15]:
# measure = "Taux de pauvreté (en\xa0%)"

# tmp = pd.merge(gdf, dd, left_on="code", right_on="Code du département")
# matplotlib.rcParams["figure.dpi"] = 200

# fig, ax = plt.subplots()

# tmp.plot(ax=ax, edgecolor="#222222", cmap="plasma", linewidth=0.3, column=measure, legend=True)
# # color=aa["color"], 

# ax.axis("off")
# fig.tight_layout()

In [16]:
# df_gini2 = pd.read_excel("/Users/cnavarreteliz/Downloads/indic-struct-distrib-revenu-2016-SUPRA/FILO2016_DISP_DEP.xls",
#                          sheet_name="ENSEMBLE", header=5)

# df_gini2 = df_gini2[["CODGEO", "GI16", "RD", "S80S2016"]]
# df_gini2

In [17]:
# df_pop = df.groupby("Code du département").agg({"Inscrits": "sum", "Abstentions": "sum"}).reset_index()
# df_pop["Abstention Rate"] = df_pop["Abstentions"] / df_pop["Inscrits"]
# df_pop

In [18]:

# df_model = pd.merge(dd, df_gini2, left_on="Code", right_on="CODGEO")
# df_model = pd.merge(df_model, df_pop, on="Code du département")
# df_model["Taux de pauvreté (en\xa0%)"] = df_model["Taux de pauvreté (en\xa0%)"] / 100
# df_model["Inscrits Log"] = np.log10(df_model["Inscrits"])
# df_model.corr()

In [19]:
# Y = df_model["Weight"]
# X = df_model[["Taux de pauvreté (en\xa0%)", "GI16", "Inscrits Log", "Abstention Rate"]]
# X = sm.add_constant(X)
# model = sm.OLS(Y,X)
# results = model.fit(cov_type="hc2")

# print(results.summary())

In [20]:
# measure = "Weight"

# data1 = df_1st_round.groupby(["Code du département", "Candidate"]).agg({measure: "std"}).reset_index()

# tmp = pd.merge(gdf, data1, left_on="code", right_on="Code du département")
# matplotlib.rcParams["figure.dpi"] = 200

# tmp = tmp[tmp["Candidate"] == "Jean-Luc MÉLENCHON"].copy()

# fig, ax = plt.subplots()

# tmp.plot(ax=ax, edgecolor="#222222", cmap="plasma", linewidth=0.3, column=measure, legend=True)
# # color=aa["color"], 

# ax.axis("off")
# fig.tight_layout()

In [21]:
# data1 = df_1st_round.groupby(["Code du département", "Candidate"]).agg({"Weight": "std"}).reset_index()

# aa = data1.groupby(["Code du département", "Candidate"]).agg({"Weight": "sum"})
# aa = aa.sort_values(["Code du département", "Weight"], ascending=[False, False])
# aa = aa.groupby(["Code du département"]).head(1).reset_index()

# matplotlib.rcParams["figure.dpi"] = 200

# # tmp = tmp[tmp["Candidate"] == "Marine LE PEN"].copy()

# tmp = pd.merge(gdf, aa, left_on="code", right_on="Code du département")
# tmp["color"] = tmp["Candidate"].replace({
#     "Nathalie ARTHAUD": "#aa0201",
#     "Fabien ROUSSEL": "#dd0200",
#     "Emmanuel MACRON": "#ffd601",
#     "Jean LASSALLE": "#adc0fd",
#     "Marine LE PEN": "#014a76",
#     "Éric ZEMMOUR": "#0b0a66",
#     "Jean-Luc MÉLENCHON": "#c9462b",
#     "Anne HIDALGO": "#ed1651",
#     "Yannick JADOT": "#79b31e",
#     "Valérie PÉCRESSE": "#0066cc",
#     "Philippe POUTOU": "#c00b1e",
#     "Nicolas DUPONT-AIGNAN": "#0587cc"
# })

# fig, ax = plt.subplots()

# tmp.plot(ax=ax, edgecolor="#222222", linewidth=0.3, color=tmp["color"], 
#          column="Candidate", legend=True,
#         legend_kwds={'loc': 'center left', "bbox_to_anchor": (1, 0.5)})



# ax.axis("off")
# fig.tight_layout()

In [22]:
# data1 = df_filtered.groupby(["Code du département", "Candidate"]).agg({"Votes": "mean"}).reset_index()

# aa = data1.groupby(["Code du département", "Candidate"]).agg({"Votes": "sum"})
# aa = aa.sort_values(["Code du département", "Votes"], ascending=[False, False])
# aa = aa.groupby(["Code du département"]).head(1).reset_index()

# matplotlib.rcParams["figure.dpi"] = 200

# # tmp = tmp[tmp["Candidate"] == "Marine LE PEN"].copy()

# tmp = pd.merge(gdf, aa, left_on="code", right_on="Code du département")
# tmp["color"] = tmp["Candidate"].replace({
#     "Nathalie ARTHAUD": "#aa0201",
#     "Fabien ROUSSEL": "#dd0200",
#     "Emmanuel MACRON": "#ffd601",
#     "Jean LASSALLE": "#adc0fd",
#     "Marine LE PEN": "#014a76",
#     "Éric ZEMMOUR": "#0b0a66",
#     "Jean-Luc MÉLENCHON": "#c9462b",
#     "Anne HIDALGO": "#ed1651",
#     "Yannick JADOT": "#79b31e",
#     "Valérie PÉCRESSE": "#0066cc",
#     "Philippe POUTOU": "#c00b1e",
#     "Nicolas DUPONT-AIGNAN": "#0587cc"
# })

# fig, ax = plt.subplots()

# tmp.plot(ax=ax, edgecolor="#222222", color=tmp["color"], linewidth=0.3, column="Candidate", legend=True)
# # color=aa["color"], 

# ax.axis("off")
# fig.tight_layout()

In [23]:
# import matplotlib.pyplot as plt  # Graphics
# import numba
# import seaborn                   # Graphics
# import geopandas                 # Spatial data manipulation
# import rioxarray                 # Surface data manipulation
# import xarray                    # Surface data manipulation

# from matplotlib import colors
# from pysal.lib import weights    # Spatial weights
# import contextily                # Background tiles



In [24]:
# from esda.moran import Moran

In [25]:
# import pysal as ps

In [26]:
# db = pd.merge(tmp, df_model, left_on="code", right_on="Code")
# db.head()

In [27]:
# feature = "Weight" #"Taux de pauvreté (en\xa0%)" # "GI16"
# w = weights.Queen.from_dataframe(db)

# # Row-standardization
# w.transform = "R"

# db[f"w_{feature}"] = weights.spatial_lag.lag_spatial(w, db[feature])

# db[f"{feature}_std"] = (db[feature] - db[feature].mean() )
# db[f'w_{feature}_std'] = weights.spatial_lag.lag_spatial(w, db[f"{feature}_std"])#( db['w_Weight'] - db['Weight'].mean() )

# fig, ax = plt.subplots()
# matplotlib.rcParams["figure.dpi"] = 200

# db.plot(ax=ax, edgecolor="#222222", cmap="plasma", linewidth=0.3, column=f"w_{feature}", legend=True)
# # color=aa["color"], 

# ax.axis("off")
# fig.tight_layout()

In [28]:
# fig, ax = plt.subplots(figsize=(7, 7))

# g = sns.regplot(x="w_GI16", y="w_Weight_x", data=db)
# matplotlib.rcParams["figure.dpi"] = 72

# g.set_ylabel("Spatial Lag Disagreements")
# g.set_yscale("log")
# g.set_xlabel("Spatial Lag GINI")

In [29]:
# # Setup the figure and axis
# f, ax = plt.subplots(1, figsize=(6, 6))
# # Plot values
# seaborn.regplot(
#     x='Weight_std', y='w_Weight_std', data=db, ci=None
# )
# # Add vertical and horizontal lines
# plt.axvline(0, c='k', alpha=0.5)
# plt.axhline(0, c='k', alpha=0.5)
# # Add text labels for each quadrant
# # plt.text(20, 5, "HH", fontsize=25, c='r')
# # plt.text(12, -11, "HL", fontsize=25, c='r')
# # plt.text(-20, 8.0, "LH", fontsize=25, c='r')
# # plt.text(-25, -11.0, "LL", fontsize=25, c='r')
# # Display
# plt.show()

In [30]:
# datax = df_filtered.groupby(["Code du département", "Candidate"]).agg({"Votes": "sum"}).reset_index()